# ETL_EDA    
This file records the process of acquiring raw data, traforming them, and loading them into a MongoDB. The data are store (almost) in their raw form.

## 1. Raw Data from LAPD     
The [dataset](https://data.lacity.org/A-Safe-City/Arrest-Data-from-2010-to-Present/yru6-6re4) is a continuously updated file in csv format. It contains City of Los Angeles arrest data from 2010 to present. It can be retrieved simply by making requests without parameters. For more complicated API call, you may need to add query parameters.

In [3]:
import logging
import pandas as pd
from sodapy import Socrata
import utils

# download data
CRIME_SOURCE = "data.lacity.org"
client0 = Socrata(CRIME_SOURCE, None)
results = client0.get("yru6-6re4",where="arst_date > '2019-12-10T00:00:00.000'",limit = 1000000) # wait to be confirmed
print(results[:2])
print('...')
print(results[-2:])

[{'rpt_id': '190129845', 'report_type': 'RFC', 'arst_date': '2019-12-11T00:00:00.000', 'time': '1510', 'area': '01', 'area_desc': 'Central', 'rd': '0152', 'age': '24', 'sex_cd': 'M', 'descent_cd': 'H', 'arst_typ_cd': 'I', 'charge': '63.44B24LAM', 'dispo_desc': 'MISDEMEANOR COMPLAINT FILED', 'location': '5TH', 'crsst': 'HILL', 'lat': '34.0489', 'lon': '-118.2519', 'location_1': {'type': 'Point', 'coordinates': [-118.2519, 34.0489]}}, {'rpt_id': '191425299', 'report_type': 'RFC', 'arst_date': '2019-12-11T00:00:00.000', 'time': '1430', 'area': '14', 'area_desc': 'Pacific', 'rd': '1409', 'age': '24', 'sex_cd': 'F', 'descent_cd': 'H', 'arst_typ_cd': 'M', 'charge': '104.01A27', 'dispo_desc': 'MISDEMEANOR COMPLAINT FILED', 'location': '9600    VENICE                       BL', 'lat': '34.0249', 'lon': '-118.3959', 'location_1': {'type': 'Point', 'coordinates': [-118.3959, 34.0249]}}]
...
[{'rpt_id': '5835308', 'report_type': 'BOOKING', 'arst_date': '2019-12-31T00:00:00.000', 'time': '1010', '

In [2]:
!pip install sodapy

## 2. Raw Data to Documents/Dicts      
Datetime conversion is made and age column is changed to numeric data type. Now the data can be easily converted to a list of dicts which is what we want for the MongoDB.

In [4]:
import logging
import pandas as pd
from sodapy import Socrata
import utils
CRIME_SOURCE = "data.lacity.org"
DOWNLOAD_PERIOD = 15         # second
logger = logging.Logger(__name__)
utils.setup_logger(logger, 'data.log')


def download_crime(url=CRIME_SOURCE, start_date = '2019-12-08T00:00:00.000'):
    """Returns records from `CRIME_SOURCE` that includes crime and arrestee information.
    """
    client = Socrata(url, None)
    results = client.get("yru6-6re4",where=f"arst_date >= '{start_date}'", limit = 10000) # wait to be confirmed
    return results

In [5]:
from datetime import datetime

df = pd.DataFrame.from_records(results)
df['arst_date'] = pd.to_datetime(df['arst_date'])
# df['hour'] = df['time'].apply(lambda x: datetime.strptime(x, '%H%M').time().hour)
#df['month'] = df['arst_date'].apply(lambda x: datetime.strptime(x, '%Y%m').time())
df['age'] = pd.to_numeric(df['age'])
df.head()

,rpt_id,report_type,arst_date,time,area,area_desc,rd,age,sex_cd,descent_cd,...,lat,lon,location_1,bkg_date,bkg_time,bgk_location,bkg_loc_cd,chrg_desc,chrg_grp_cd,grp_description
0,190129845,RFC,2019-12-11,1510,01,Central,0152,24,M,H,...,34.0489,-118.2519,"{'type': 'Point', 'coordinates': [-118.2519, 3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,191425299,RFC,2019-12-11,1430,14,Pacific,1409,24,F,H,...,34.0249,-118.3959,"{'type': 'Point', 'coordinates': [-118.3959, 3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,190129848,RFC,2019-12-11,0950,01,Central,0138,59,M,B,...,34.0468,-118.2415,"{'type': 'Point', 'coordinates': [-118.2415, 3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,191425300,RFC,2019-12-11,1430,14,Pacific,1409,23,F,W,...,34.0249,-118.3959,"{'type': 'Point', 'coordinates': [-118.3959, 3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,190129844,RFC,2019-12-11,1450,01,Central,0152,43,M,B,...,34.0489,-118.2519,"{'type': 'Point', 'coordinates': [-118.2519, 3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df['location_1']

0       {'type': 'Point', 'coordinates': [-118.2519, 3...
1       {'type': 'Point', 'coordinates': [-118.3959, 3...
2       {'type': 'Point', 'coordinates': [-118.2415, 3...
3       {'type': 'Point', 'coordinates': [-118.3959, 3...
4       {'type': 'Point', 'coordinates': [-118.2519, 3...
                              ...                        
2670    {'type': 'Point', 'coordinates': [-118.4789, 3...
2671    {'type': 'Point', 'coordinates': [-118.2035, 3...
2672    {'type': 'Point', 'coordinates': [-118.6233, 3...
2673    {'type': 'Point', 'coordinates': [-118.4052, 3...
2674    {'type': 'Point', 'coordinates': [-118.4465, 3...
Name: location_1, Length: 2675, dtype: object

## 3. Upsert MongoDB         
To avoid duplicate data entries between each run, the de-duplication mechanism makes sure that only a new record with a different report_id (rpt_id) will be inserted to the database. 

In [30]:
import pymongo
client = pymongo.MongoClient()

In [33]:
#import time
db = client.get_database("crime")
collection = db.get_collection("crime")
update_count = 0
count = 0
for record in df.to_dict('records'):
    #start = time.time()
    result = collection.replace_one(
        filter = {'rpt_id': record['rpt_id']},    # locate the document if exists
        replacement = record,                         # latest document
        upsert=True)
    if result.matched_count > 0:
        update_count += 1
print(f"rows={df.shape[0]}, update={update_count}, "
      f"insert={df.shape[0]-update_count}")

rows=495, update=495, insert=0
